# 1 - Fine Tuning

<br>
<br>

<img src="https://raw.githubusercontent.com/Hack-io-AI/ai_images/main/openai_fine_tuning.webp" style="width:400px;"/>


<h1>Tabla de Contenidos<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1---¿Qué-es-Fine-Tuning?" data-toc-modified-id="1---¿Qué-es-Fine-Tuning?-1">1 - ¿Qué es Fine Tuning?</a></span></li><li><span><a href="#2---Configuración" data-toc-modified-id="2---Configuración-2">2 - Configuración</a></span></li><li><span><a href="#3---Preparación-de-datos" data-toc-modified-id="3---Preparación-de-datos-3">3 - Preparación de datos</a></span></li><li><span><a href="#4---Subida-de-archivos" data-toc-modified-id="4---Subida-de-archivos-4">4 - Subida de archivos</a></span></li><li><span><a href="#5---Fine-Tuning" data-toc-modified-id="5---Fine-Tuning-5">5 - Fine Tuning</a></span><ul class="toc-item"><li><span><a href="#5.1---Status-Check" data-toc-modified-id="5.1---Status-Check-5.1">5.1 - Status Check</a></span></li></ul></li><li><span><a href="#6---Inferencia" data-toc-modified-id="6---Inferencia-6">6 - Inferencia</a></span></li></ul></div>

## 1 - ¿Qué es Fine Tuning?

El fine-tuning, o ajuste fino, es una técnica en el campo del aprendizaje automático que implica tomar un modelo previamente entrenado en una gran cantidad de datos generales y ajustarlo, reentrenarlo, con un conjunto de datos específico para una tarea particular. Esto permite que el modelo sea más preciso y eficaz en esa tarea específica sin tener que entrenar un modelo desde cero, lo cual puede ser costoso y llevar mucho tiempo. 


**Características del Fine-Tuning**


1. **Uso de modelos preentrenados**: El fine-tuning comienza con un modelo que ya ha sido entrenado en una gran cantidad de datos generales. Estos modelos preentrenados ya han aprendido representaciones útiles y características básicas del lenguaje o imágenes, dependiendo del tipo de modelo.


2. **Entrenamiento en datos específicos**: Después de tener el modelo preentrenado, se le reentrena, ajusta fino, con un conjunto de datos más pequeño y específico para la tarea en cuestión. Esto puede incluir datos específicos de un dominio, como textos médicos, datos financieros o cualquier otro tipo de información especializada.


3. **Ajuste de parámetros**: Durante el fine-tuning, solo algunos de los parámetros del modelo pueden ser ajustados, mientras que otros se mantienen congelados, sin cambios. Esto ayuda a preservar el conocimiento general adquirido durante el preentrenamiento mientras se adapta a las especificidades del nuevo conjunto de datos.


<br>


**Ventajas del Fine-Tuning**

+ Eficiencia en tiempo y recursos. Es mucho más rápido y menos costoso ajustar un modelo preentrenado que entrenar uno desde cero.


+ Mejora en la precisión. Permite que el modelo sea altamente preciso en tareas específicas gracias a la especialización proporcionada por los datos de ajuste fino.


+ Flexibilidad. Puede ser aplicado a una amplia gama de tareas y dominios, haciendo que los modelos sean adaptables a diferentes necesidades.


<br>


**Desafíos del Fine-Tuning**

+ Sobrecarga de datos específicos. Existe el riesgo de que el modelo se ajuste demasiado a los datos específicos y pierda su capacidad de generalización.


+ Necesidad de datos etiquetados. Requiere un conjunto de datos bien etiquetado y relevante para la tarea específica, lo cual puede ser difícil y costoso de obtener.


+ Ajuste de hiperparámetros. Encontrar los hiperparámetros correctos (como la tasa de aprendizaje) para el ajuste fino puede ser complejo y requiere experimentación.


<br>


**Proceso de Fine-Tuning**


1. **Configuración**: Cargar nuestro conjunto de datos y filtrar para centrarse en un solo dominio para el ajuste fino.


2. **Preparación de datos**: Preparar nuestros datos para el ajuste fino creando ejemplos de entrenamiento y validación, y subirlos al endpoint de Archivos.


3. **Ajuste fino**: Crear nuestro modelo ajustado finamente.


4. **Inferencia**: Usar nuestro modelo ajustado finamente para hacer inferencia en nuevas entradas.

## 2 - Configuración

In [ ]:
# importamos librerías, API KEY e iniciamos cliente

import os                           
from dotenv import load_dotenv 
import openai as ai
import pandas as pd


load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

cliente = ai.OpenAI()

El ajuste fino funciona mejor cuando se enfoca en un dominio particular. Es importante asegurarse de que nuestro conjunto de datos sea lo suficientemente específico para que el modelo pueda aprender, pero lo suficientemente general para que no se pierdan ejemplos no vistos. Teniendo esto en cuenta, hemos extraído un subconjunto del conjunto de datos RecipesNLG para que solo contenga documentos de [cookbooks](https://cookbooks.com/).

## 3 - Preparación de datos

In [ ]:
# carga del csv

data = pd.read_csv('../../files/cookbook_recipes_nlg_10k.csv')

data.head()

Comenzaremos preparando nuestros datos. Al ajustar finamente con el formato `ChatCompletion`, cada ejemplo de entrenamiento es una lista simple de mensajes. Por ejemplo, una entrada podría verse así:

```python
[{'role': 'system',
  'content': 'You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided.'},

 {'role': 'user',
  'content': 'Title: No-Bake Nut Cookies\n\nIngredients: ["1 c. firmly packed brown sugar", "1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 c. broken nuts (pecans)", "2 Tbsp. butter or margarine", "3 1/2 c. bite size shredded rice biscuits"]\n\nGeneric ingredients: '},

 {'role': 'assistant',
  'content': '["brown sugar", "milk", "vanilla", "nuts", "butter", "bite size shredded rice biscuits"]'}]
```

Durante el proceso de entrenamiento, esta conversación se dividirá, con la última entrada siendo la finalización que el modelo producirá, y el resto de los mensajes actuando como el prompt. Tengamos en cuenta esto al construir nuestros ejemplos de entrenamiento: si nuestro modelo actuará en conversaciones de varios turnos, proporcionaremos ejemplos representativos para que no tenga un rendimiento deficiente cuando la conversación comience a expandirse.

Actualmente hay un límite de 4096 tokens para cada ejemplo de entrenamiento. Cualquier cosa más larga que esto se truncará a 4096 tokens.

In [ ]:
mensaje_sistema = 'You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided.'


def crear_mensaje_usuario(fila: object) -> str:
    
    """
    Función para crear el mensaje del usuario.
    
    Params:
    fila: fila de un pandas dataframe
    
    Return:
    string 
    """
    
    return f"Title: {fila['title']}\n\nIngredients: {fila['ingredients']}\n\nGeneric ingredients: "



def preparar_ejemplo_conversacion(fila):
    
    """
    Función para perparar la conversacioncon el chat.
    
    Params:
    fila: fila de un pandas dataframe
    
    Return:
    dict 
    
    """
    
    return {'messages': [{'role': 'system', 'content': mensaje_sistema},
                         {'role': 'user', 'content': crear_mensaje_usuario(fila)},
                         {'role': 'assistant', 'content': fila['NER']}
                        ]
           }


In [ ]:
preparar_ejemplo_conversacion(data.iloc[0])

Ahora hagamos esto para un subconjunto del conjunto de datos que utilizaremos como datos de entrenamiento. Podemos comenzar con 30-50 ejemplos bien seleccionados. Deberíamos ver que el rendimiento continúa escalando linealmente a medida que aumentamos el tamaño del conjunto de entrenamiento, pero también tardarán más.

In [ ]:
# usamos las primeras 100 filas para entrenar
train_data = data.loc[0:100]

# le aplicamos la preparacion de conversacion, pasamos a lista porque el objetivo es tener un json
train_data = train_data.apply(preparar_ejemplo_conversacion, axis=1).tolist()

for ejemplo in train_data[:5]:
    print(ejemplo)

Además de los datos de entrenamiento, también podemos proporcionar opcionalmente datos de validación, que se utilizarán para asegurarse de que el modelo no se sobreajuste al conjunto de entrenamiento.


In [ ]:
# 100 filas de validacion
val_data = data.loc[101:200]

val_data = val_data.apply(preparar_ejemplo_conversacion, axis=1).tolist()

Ahora que tenemos listos los datos de entrenamiento y de validación, los guardamos en un archivo JSON, con extensión `.jsonl`.

In [ ]:
import json

def escribir_json(data: list, archivo: str) -> None:
    
    with open(archivo, 'w') as f:
        
        for d in data:
            salida = json.dumps(d) + '\n'
            f.write(salida)

In [ ]:
# guardado de archivos

escribir_json(train_data, 'recipe_finetune_train.jsonl')

escribir_json(val_data, 'recipe_finetune_validacion.jsonl')

## 4 - Subida de archivos

Una vez que tenemos los archivos de datos preparados, tenemos que subirlos al endpoint de OpanAI para ajustar el modelo.

In [ ]:
# funcion para subir archivos

def subir_archivo(archivo: str, proposito: str = 'fine-tune') -> str:
    """
    Esta funcion sube archivos a OpenAI
    
    Params:
    archivo: string, ruta al archivo que queremos subir
    proposito: string, proposito del archivo, en este caso fine-tune por defecto
    
    Return:
    string con el ID del archivo
    """
    
    global cliente
    
    with open(archivo, 'rb') as f:
        respuesta = cliente.files.create(file=f, purpose=proposito)
        
    return respuesta.id

In [ ]:
# subida archivos

train_id = subir_archivo('recipe_finetune_train.jsonl')

val_id = subir_archivo('recipe_finetune_validacion.jsonl')

In [ ]:
print('Training ID:', train_id)

print('Validacion ID:', val_id)

## 5 - Fine Tuning

Ahora podemos crear nuestro trabajo de fine tuning con los archivos generados y un sufijo opcional para identificar el modelo. La respuesta contendrá un ID que podemos usar para obtener actualizaciones sobre el trabajo.

Nota: Los archivos deben ser procesados primero por nuestro sistema, por lo que podríamos recibir un error de `File not ready` ("Archivo no listo"). En ese caso, simplemente lo intentaremos de nuevo unos minutos más tarde.

In [ ]:
respuesta = cliente.fine_tuning.jobs.create(training_file=train_id,
                                            validation_file=val_id,
                                            model='gpt-4o-mini-2024-07-18',
                                            suffix='recipe-ner')

job_id = respuesta.id

print('Job ID:', respuesta.id)
print('Status:', respuesta.status)

### 5.1 - Status Check

Podemos hacer una solicitud `GET` al endpoint https://api.openai.com/v1/alpha/fine-tunes para listar nuestros trabajos de ajuste fino. En este caso, querremos verificar que el ID obtenido del paso anterior tenga el estado "succeeded".

Una vez completado, podemos usar los archivos de resultado para muestrear los resultados del conjunto de validación y utilizar el ID del parámetro fine_tuned_model para invocar nuestro modelo entrenado.

In [ ]:
respuesta = cliente.fine_tuning.jobs.retrieve(job_id)


print('Job ID:', respuesta.id)
print('Status:', respuesta.status)
print('Tokens Entrenados:', respuesta.trained_tokens)

Podemos hacer un seguimiento del progreso del ajuste fino con el endpoint de eventos. Podemos ejecutar la siguiente celda varias veces hasta que el ajuste fino esté listo.

In [ ]:
respuesta = cliente.fine_tuning.jobs.list_events(job_id)

eventos = respuesta.data
eventos.reverse()

for e in eventos:
    print(e.message)

Una vez terminado, podemos obtener un ID de modelo ajustado a partir del trabajo:

In [ ]:
respuesta = cliente.fine_tuning.jobs.retrieve(job_id)

fine_tune_id = respuesta.fine_tuned_model


if fine_tune_id is None:
    
    raise RuntimeError('ID no encontrado. El Fine Tuning no ha sido completado.')

    
print('ID Modelo Ajustado:', fine_tune_id)

## 6 - Inferencia

El último paso es usar nuestro modelo ajustado para inferencia, simplemente llamamos a `ChatCompletions` con el nombre de nuestro nuevo modelo ajustado, completando el parámetro model.

In [ ]:
# seleccion fila de testeo

test_data = data.loc[201:300]

test = test_data.iloc[0]

In [ ]:
# creacion de mensaje

mensajes_test = []

mensajes_test.append({'role': 'system', 'content': mensaje_sistema})

mensaje_usuario = crear_mensaje_usuario(test)

mensajes_test.append({'role': 'user', 'content': mensaje_usuario})

print(mensajes_test)

In [ ]:
# respuesta del modelo

respuesta = cliente.chat.completions.create(model=fine_tune_id, 
                                            messages=mensajes_test, 
                                            temperature=0, 
                                            max_tokens=500)


In [ ]:
print(respuesta.choices[0].message.content)